<a href="https://colab.research.google.com/github/Xiaolumang/breaking-captcha/blob/main/tree/rnn_model/Teams_T3_2022/Audio_CAPTCHA/Fenglu_Breaking_audio/Botdetect_Audio_CAPTCHA_Breaking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import librosa
import librosa.display
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math
import torch.nn as nn
import matplotlib.pyplot as plt
import string
import io
import os
import unicodedata
import string
import glob

import torch
import random

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
input_size = 13
hidden_size = 256
#num_layers 
learning_rate = 0.001
num_classes = 62  #26 + 26 + 10

batch_size = 64
# Use Mel-frequency ceptrum coefficients from wav signals
def extract_mfcc(audio_path):
  x, sr = librosa.load(audio_path)
  mfccs = librosa.feature.mfcc(x, sr=sr,n_mfcc=input_size)
  return mfccs.T

In [ ]:
t = string.ascii_letters+string.digits
lookup = dict()
for x, y in enumerate([*t]):
  lookup[y] = x

In [ ]:
class CAPTCHADataset(Dataset):
  def __init__(self):
    super().__init__()
    file_path = '/content/drive/MyDrive/Deakin_2022_T3/SIT782_CapstoneB/CAPTCHAs/botdetect'
    audio_dataset = []
    for x in glob.glob(os.path.join(file_path,'*.wav')):
      mfcc = extract_mfcc(x)
#  print(mfcc.shape)
      audio_dataset.append(torch.Tensor(mfcc))
    self.x_data = torch.nn.utils.rnn.pad_sequence(audio_dataset,batch_first=True,padding_value=0)
    with open(os.path.join(file_path, 'labels.txt')) as f:
      self.y_data = [lookup[x.strip()] for x in f.readlines()]
    self.min_len = min(len(self.x_data),len(self.y_data))
    
  def __getitem__(self, index):
      sample = self.x_data[index], self.y_data[index]
      return sample
    
  def __len__(self):
      return self.min_len


In [ ]:
dataset = CAPTCHADataset()

In [ ]:
data_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True,num_workers=2)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 100
num_layers=2
num_iterations = math.ceil(len(dataset)/batch_size)
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size,num_layers, num_classes):
    super().__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size, num_classes )

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

    out, _ = self.lstm(x,(h0,c0))
    out = out[:, -1, :]

    out = self.fc(out)
    return out


In [ ]:
model = LSTM(input_size, hidden_size, num_layers, num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(data_loader):
    
    data = data.to(device=device).squeeze(1)
    targets = targets.to(device=device)
    scores = model(data)
    loss = criterion(scores, targets)
    if (batch_idx) % 20 == 0:
      print(batch_idx)
      print(f'epoch {epoch+1}/{num_epochs}, step {batch_idx+1}/{num_iterations} loss {loss}')
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0
epoch 1/100, step 1/1 loss 4.136926174163818
0
epoch 2/100, step 1/1 loss 4.091273784637451
0
epoch 3/100, step 1/1 loss 4.043126583099365
0
epoch 4/100, step 1/1 loss 3.98577618598938
0
epoch 5/100, step 1/1 loss 3.912424087524414
0
epoch 6/100, step 1/1 loss 3.800797700881958
0
epoch 7/100, step 1/1 loss 3.6065165996551514
0
epoch 8/100, step 1/1 loss 3.1911020278930664
0
epoch 9/100, step 1/1 loss 2.5667312145233154
0
epoch 10/100, step 1/1 loss 2.28601336479187
0
epoch 11/100, step 1/1 loss 2.20454478263855
0
epoch 12/100, step 1/1 loss 2.154461622238159
0
epoch 13/100, step 1/1 loss 2.1384928226470947
0
epoch 14/100, step 1/1 loss 2.130995273590088
0
epoch 15/100, step 1/1 loss 2.11687970161438
0
epoch 16/100, step 1/1 loss 2.100524425506592
0
epoch 17/100, step 1/1 loss 2.088300943374634
0
epoch 18/100, step 1/1 loss 2.1676769256591797
0
epoch 19/100, step 1/1 loss 2.151745080947876
0
epoch 20/100, step 1/1 loss 2.0733630657196045
0
epoch 21/100, step 1/1 loss 2.068571329116821